In [1]:
import pandas as pd
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils  import to_categorical
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import *

In [2]:
df="/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TRAIN"

In [3]:
df_label=pd.read_csv('/kaggle/input/blood-cells/dataset2-master/dataset2-master/labels.csv')

In [4]:
df_label

,Unnamed: 0,Image,Category
0,NaN,0,NEUTROPHIL
1,NaN,1,NEUTROPHIL
2,NaN,2,NEUTROPHIL
3,NaN,3,NEUTROPHIL
4,NaN,4,NEUTROPHIL
...,...,...,...
406,NaN,406,NaN
407,NaN,407,NEUTROPHIL
408,NaN,408,NEUTROPHIL
409,NaN,409,NEUTROPHIL


In [5]:
c=cv2.imread("/kaggle/input/blood-cells/dataset2-master/dataset2-master/images/TRAIN/EOSINOPHIL/_0_1414.jpeg")
c.shape

(240, 320, 3)

In [6]:
new_data=[]
label=[]
for cur,_,filename in os.walk(df):
    for file in filename:
        data=os.path.join(cur,file)
        label_name=os.path.basename(os.path.dirname(data))
        cv=cv2.imread(data)
        cv_re=cv2.resize(cv,(256,256))
        new_data.append(cv_re)
        label.append(label_name)
new_data=np.array(new_data)
# label=np.array(label)

In [7]:
new_data.shape

(9957, 256, 256, 3)

In [8]:
# label.shape

In [9]:
one_hot=LabelEncoder()
one=one_hot.fit_transform(label)
one_categ=to_categorical(one)
one_categ.shape

(9957, 4)

In [10]:
datagen=ImageDataGenerator(
rescale=1.0/255,
rotation_range=40,
shear_range=0.2,
zoom_range=0.2,
width_shift_range=0.2,
height_shift_range=0.2,
fill_mode="nearest",
validation_split=0.2
)

In [11]:
X_train,X_test,Y_train,Y_test=train_test_split(new_data,one_categ,test_size=0.2,random_state=42)

In [12]:
train_data=datagen.flow(
X_train,
# class_mode=None,
# target_size=(256, 256),
batch_size=32,
# subset="training"
)
val_data=datagen.flow(
X_test,
# target_size=(256, 256),
# class_mode=None,
batch_size=32,
# subset="validation"
)

In [13]:
img2=next(train_data)
print(img2.shape)
# print(lab2.shape)

(32, 256, 256, 3)


In [14]:
# np.unique(Y_train)

# # Deep learning

In [15]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,BatchNormalization,Dropout,Input,TimeDistributed,GlobalAveragePooling2D,Conv2DTranspose
from keras.models import Model
from keras.regularizers import l2
from keras.initializers import HeNormal
from keras.callbacks import EarlyStopping

In [16]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [27]:
inputs=Input(shape=(256,256,3))
C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)

C=Conv2D(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)

M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)
C=Conv2D(64,(5,5),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(64,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)
M=MaxPooling2D(pool_size=(2,2),strides=(2,2))(B)

C=Conv2D(128,(5,5),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(M)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)




C=Conv2DTranspose(64,(3,3),strides=(2,2),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)
C=Conv2DTranspose(64,(3,3),strides=(2,2),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)
C=Conv2DTranspose(32,(5,5),strides=(2,2),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)

C=Conv2DTranspose(32,(3,3),activation="relu",padding="same",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dropout(0.25)(C)
B=BatchNormalization()(D)

outputs=Conv2D(3,(3,3),activation="sigmoid",padding="same")(B)

model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['mae'])
model.summary()


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 256, 256, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_53 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_53          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_36 (Conv2D)              │ (None, 256, 256, 32)   │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_54 (Dropout)            │ (None, 256, 256, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_54          │ (None, 256, 256, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_37 (Conv2D)              │ (None, 128, 128, 64)   │        51,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_55 (Dropout)            │ (None, 128, 128, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_55          │ (None, 128, 128, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_38 (Conv2D)              │ (None, 64, 64, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_56 (Dropout)            │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_56          │ (None, 64, 64, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 32, 32, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_39 (Conv2D)              │ (None, 32, 32, 128)    │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_57 (Dropout)            │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_57          │ (None, 32, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_23             │ (None, 64, 64, 64)     │        73,792 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_58 (Dropout)            │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_58          │ (None, 64, 64, 64)     │           25

 Total params: 477,379 (1.82 MB)

 Trainable params: 476,355 (1.82 MB)

 Non-trainable params: 1,024 (4.00 KB)

In [28]:
model.fit(X_train,X_train,epochs=50,validation_split=0.2,callbacks=[early_stopping])

Epoch 1/50


I0000 00:00:1727462512.933854    1649 service.cc:145] XLA service 0x7aa96407b5f0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727462512.934689    1649 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2024-09-27 18:42:11.249758: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[64,32,5,5]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,128,128]{3,2,1,0}, f32[32,64,128,128]{3,2,1,0}), window={size=5x5 pad=2_2x2_2}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBackwardFilter", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-27 18:42:11.280511: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.030889167s
Trying algorithm eng0{} for conv

  1/200 ━━━━━━━━━━━━━━━━━━━━ 1:59:09 36s/step - loss: 754.4271 - mae: 170.4707

I0000 00:00:1727462541.597620    1649 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 ━━━━━━━━━━━━━━━━━━━━ 93s 286ms/step - loss: 708.0251 - mae: 167.6258 - val_loss: 604.5139 - val_mae: 167.6584
Epoch 2/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - loss: 834.4133 - mae: 167.6340 - val_loss: 607.7308 - val_mae: 167.6086
Epoch 3/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - loss: 968.3403 - mae: 167.4088 - val_loss: 675.2939 - val_mae: 167.5953
Epoch 4/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 172ms/step - loss: 1100.0364 - mae: 167.4199 - val_loss: 733.5571 - val_mae: 167.5959
Epoch 5/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - loss: 1235.0958 - mae: 167.4988 - val_loss: 797.2861 - val_mae: 167.5928
Epoch 6/50
200/200 ━━━━━━━━━━━━━━━━━━━━ 34s 170ms/step - loss: 1370.3838 - mae: 167.3224 - val_loss: 822.0769 - val_mae: 167.6152
Epoch 6: early stopping


In [29]:
prediction=model.predict(X_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 5s 65ms/step


In [30]:
reconstruction_error=np.mean(np.square(X_test-prediction))
threshold=np.mean(reconstruction_error)+2*np.std(reconstruction_error)
label=np.where(reconstruction_error>threshold,0,1)
accuracy=np.mean(label==0)*100
print(accuracy)

0.0
